<a href="https://colab.research.google.com/github/zevan07/DS-Unit-1-Sprint-3-Statistical-Tests-and-Experiments/blob/master/Copy_of_LS_DS_131_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 3 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [0]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind, ttest_ind_from_stats, ttest_rel, ttest_1samp

In [5]:
! curl https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.names

1. Title: 1984 United States Congressional Voting Records Database

2. Source Information:
    (a) Source:  Congressional Quarterly Almanac, 98th Congress, 
                 2nd session 1984, Volume XL: Congressional Quarterly Inc. 
                 Washington, D.C., 1985.
    (b) Donor: Jeff Schlimmer (Jeffrey.Schlimmer@a.gp.cs.cmu.edu)
    (c) Date: 27 April 1987 

3. Past Usage
   - Publications
     1. Schlimmer, J. C. (1987).  Concept acquisition through 
        representational adjustment.  Doctoral dissertation, Department of 
        Information and Computer Science, University of California, Irvine, CA.
        -- Results: about 90%-95% accuracy appears to be STAGGER's asymptote
     - Predicted attribute: party affiliation (2 classes)

4. Relevant Information:
      This data set includes votes for each of the U.S. House of
      Representatives Congressmen on the 16 key votes identified by the
      CQA.  The CQA lists nine different types of votes: voted for, paired
      

In [31]:
# the data file does not have a header so we'll need to create one
# attribute info copy and pasted from name file
attribute_info = '''1. Class-Name: 2 (democrat, republican)
   2. handicapped-infants: 2 (y,n)
   3. water-project-cost-sharing: 2 (y,n)
   4. adoption-of-the-budget-resolution: 2 (y,n)
   5. physician-fee-freeze: 2 (y,n)
   6. el-salvador-aid: 2 (y,n)
   7. religious-groups-in-schools: 2 (y,n)
   8. anti-satellite-test-ban: 2 (y,n)
   9. aid-to-nicaraguan-contras: 2 (y,n)
  10. mx-missile: 2 (y,n)
  11. immigration: 2 (y,n)
  12. synfuels-corporation-cutback: 2 (y,n)
  13. education-spending: 2 (y,n)
  14. superfund-right-to-sue: 2 (y,n)
  15. crime: 2 (y,n)
  16. duty-free-exports: 2 (y,n)
  17. export-administration-act-south-africa: 2 (y,n)'''

# clean up attribute info to use for column headers
names = (attribute_info.replace(': 2 (y,n)', ' ')
         .replace(': 2 (democrat, republican)', ' ')
         .replace('.', ' ')
         .split())

# finish cleaning by getting rid of numbers
for x in names:
    nums = [str(x) for x in range(0, 18)]
    if x in nums:
        names.remove(x)
        
print(names)

['Class-Name', 'handicapped-infants', 'water-project-cost-sharing', 'adoption-of-the-budget-resolution', 'physician-fee-freeze', 'el-salvador-aid', 'religious-groups-in-schools', 'anti-satellite-test-ban', 'aid-to-nicaraguan-contras', 'mx-missile', 'immigration', 'synfuels-corporation-cutback', 'education-spending', 'superfund-right-to-sue', 'crime', 'duty-free-exports', 'export-administration-act-south-africa']


In [43]:
# import the csv without the first row as a header
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data', header=None)

df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


In [44]:
# verify that the names are correct
df.columns = names
df.head()

,Class-Name,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


In [45]:
# replace all 'y', 'n', and '?' values with python friendly values
df = df.replace({'y': 1, 'n': 0, '?': np.NaN})
print(df.shape)
df.head()

(435, 17)


,Class-Name,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [0]:
# create dataframes for each party
rep = df[df['Class-Name'] == 'republican']
dem = df[df['Class-Name'] == 'democrat']

## One Sample Testing

In [61]:
# after some testing, I found that republican support for this bill is high
rep['el-salvador-aid'].value_counts()

1.0    157
0.0      8
Name: el-salvador-aid, dtype: int64

In [97]:
ttest_1samp(rep['el-salvador-aid'], .9, nan_policy='omit')

Ttest_1sampResult(statistic=3.0714682478339617, pvalue=0.0024943083818825657)

- Null hypothesis: there is 90% support for the el-salvador-aid bill among conservatives. 
- Alternative hypothesis: there is non-90% republican support for this bill. 

Given the results of the above test I have failed to reject the null hypothesis 

## Two Sample Testing

In [0]:
def ttests(column):
    print('Democratic Support:', round((dem[column].mean() * 100), 1), '%')
    print('Republican Support:' , round((rep[column].mean() * 100), 1), '%')
    print('Difference:', round((rep[column].mean() - dem[column].mean()) * 100, 1), '%')
    print(ttest_ind(rep[column], dem[column], nan_policy='omit'))

In [127]:
for name in names[1:]:
    print(name)
    ttests(name)
    print(' ')
    

handicapped-infants
Democratic Support: 60.5 %
Republican Support: 18.8 %
Difference: -41.7 %
Ttest_indResult(statistic=-9.205264294809222, pvalue=1.613440327937243e-18)
 
water-project-cost-sharing
Democratic Support: 50.2 %
Republican Support: 50.7 %
Difference: 0.5 %
Ttest_indResult(statistic=0.08896538137868286, pvalue=0.9291556823993485)
 
adoption-of-the-budget-resolution
Democratic Support: 88.8 %
Republican Support: 13.4 %
Difference: -75.4 %
Ttest_indResult(statistic=-23.21277691701378, pvalue=2.0703402795404463e-77)
 
physician-fee-freeze
Democratic Support: 5.4 %
Republican Support: 98.8 %
Difference: 93.4 %
Ttest_indResult(statistic=49.36708157301406, pvalue=1.994262314074344e-177)
 
el-salvador-aid
Democratic Support: 21.6 %
Republican Support: 95.2 %
Difference: 73.6 %
Ttest_indResult(statistic=21.13669261173219, pvalue=5.600520111729011e-68)
 
religious-groups-in-schools
Democratic Support: 47.7 %
Republican Support: 89.8 %
Difference: 42.1 %
Ttest_indResult(statistic=9.

Problem 2: 
- Null Hypothesis: the mean of democratic support for the education spending bill is equal to the mean of republican support for the bill. 
-Alternative Hypothesis. the level of support is not equal.

Given the result of the above test, I would reject the null hypothesis that there is the same level of support for the education spending bill among democrats and republicans with a confidence level at the 99% confidence level. There is 72.6% more support among republicans for the bill. 

Problem 3:
- Null Hypothesis: the mean of democratic support for the education spending bill is equal to the mean of republican support for the bill. 
-Alternative Hypothesis. the level of support is not equal.

Given the result of the above test, I would reject the null hypothesis that there is the same level of support for the satellite test ban treaty bill among democrats and republicans with a confidence level at the 99% confidence level. There is 53.1% more support among democrats for the bill.


Problem 4:
- Null Hypothesis: the mean of democratic support for the water project cost sharing bill is equal to the mean of republican support for the bill. 
- Alternative Hypothesis: the level of support is not equal. 

Given the results of the above test, I have failed to reject the null hypothesis that the level of support for the water cost sharing bill is equal between democrats and republicans for the water-project-cost-sharing bill (p = .93). 